In [ ]:
# !pip install datasets==3.6.0
# !pip install transformers
# !pip install tf-keras
# !pip install transformers[torch]
# !pip install wandb
# !pip install librosa
# !pip install numpy==1.26.4

from huggingface_hub import login
login("")

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
import torch
import warnings

logging.basicConfig(level=logging.INFO)
warnings.filterwarnings('ignore')
logging.getLogger("pyngrok").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("torch").setLevel(logging.ERROR)
logger = logging.getLogger(__name__)

import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

Using device: cuda
PyTorch version: 2.7.0
CUDA device: NVIDIA A100-SXM4-40GB


In [4]:
from transformers import AutoProcessor, AutoModelForCTC, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import librosa
import numpy as np
import re
import gc
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from tqdm import tqdm
import copy

2025-07-23 12:43:53.431905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753274633.516370   63303 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753274633.541503   63303 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753274633.706901   63303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753274633.706925   63303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753274633.706927   63303 computation_placer.cc:177] computation placer alr

In [ ]:
from datasets import load_dataset, Dataset, Audio

ds = load_dataset("Elormiden/MilaMou_Cypriot_Dataset")

In [6]:
train_ds = ds['train']
eval_ds = ds['validation']

In [7]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3-turbo")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3-turbo")

In [8]:
def sampling_map(array):
    sr = array['audio']['sampling_rate']
    tr = 16000
    if sr != tr:  
        resample_array = librosa.resample(array['audio']['array'], orig_sr=sr, target_sr=tr)
        array['audio'] = {
            'path': array['audio']['path'],
            'array': resample_array,
            'sampling_rate': tr
        }
    return array

In [9]:
reforged_train = [sampling_map(sample) for sample in tqdm(train_ds, desc="Resampling")]
reforged_eval = [sampling_map(sample) for sample in tqdm(eval_ds, desc="Resampling")]

Resampling: 100%|██████████| 1708/1708 [00:05<00:00, 308.05it/s]


In [10]:
# model

In [11]:
def tokens_working_processes_wac2vec2(rf_ds): # Wac2vec2 architecture
    audio_arrays = [sample["audio"]["array"] for sample in rf_ds]
    sentences = [sample["sentence"] for sample in rf_ds]
    pronunciation = [sample["pronunciation"] for sample in rf_ds]

    inputs = processor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        max_length=16000,
        truncation=True
    )

    # Word registry fix, in case some models don't support lower registry letters (common issue with Jonathas greek model)
    sentences_upper = [sentence.upper() for sentence in sentences]
    pronunciation_upper = [pronun.upper() for pronun in pronunciation]

    labels = processor.tokenizer(
        sentences_upper, 
        padding='max_length',
        max_length=512,
        truncation=True
    )

    pronounce_labels = processor.tokenizer(
        pronunciation_upper,
        padding='max_length',
        max_length=512,
        truncation=True
    )

    labels_ids = labels["input_ids"]
    labels_ids = torch.tensor(labels_ids)  
    labels_ids[labels_ids == 0] = -100

    pronounce_labels_ids = pronounce_labels["input_ids"]
    pronounce_labels_ids = torch.tensor(pronounce_labels_ids)  
    pronounce_labels_ids[pronounce_labels_ids == 0] = -100

    return {
        **inputs,
        "labels": labels_ids,
        # "pronounce_labels": pronounce_labels_ids # Greek model does not recognize english tokens -> therefore we will skip pronunciation for now
    }

In [34]:
def tokens_working_processes_whisper(rf_ds): # Whisper architecture
    audio_arrays = [sample["audio"]["array"] for sample in rf_ds]
    sentences = [sample["sentence"] for sample in rf_ds]

    inputs = processor(
        audio_arrays,
        sampling_rate=16000,
        padding='longest',
        max_length=256,
        truncation=False
    )

    # Word registry fix, in case some models don't support lower registry letters (common issue with Jonathas greek model)
    sentences_upper = [sentence.upper() for sentence in sentences]

    labels = processor.tokenizer(
        sentences_upper, 
        padding='max_length',
        max_length=448,
        truncation=True
    )

    labels_ids = labels["input_ids"]
    labels_ids = torch.tensor(labels_ids, dtype=torch.int32)
    labels_ids[labels_ids == 0] = -100

    return {
        **inputs,
        "labels": labels_ids.numpy().astype(np.int32)
    }

In [ ]:
processed_data_train = tokens_working_processes_whisper(reforged_train)
processed_data_eval = tokens_working_processes_whisper(reforged_eval)

In [20]:
processed_data_train['labels'][25]

array([50258, 50364,   138,   239,   138,   251,   138,    97,   138,
         247,   138,   250,   138,   243,   138,    97,   138,   102,
         138,   254,   138,   232,   138,    96,   138,   243,   138,
         247,   138,    96, 24834,   138,   247,   138,   239, 48924,
         138,   239,   138,    94,   138,   228, 20838,   138,   244,
         138,   239,   138,   232, 24834,   138,   234,   138,   251,
         138,   253,   138,    96, 26408,   138,   253,   138,    98,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,
       50257, 50257,

In [22]:
train_hf = Dataset.from_dict(processed_data_train)
eval_hf = Dataset.from_dict(processed_data_eval)

In [23]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor ([`WhisperProcessor`])
            The processor used for processing the data.
        decoder_start_token_id (`int`)
            The begin-of-sentence of the decoder.
        forward_attention_mask (`bool`)
            Whether to return attention_mask.
    """

    processor: Any
#     decoder_start_token_id: int
#     forward_attention_mask: bool

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        model_input_name = self.processor.model_input_names[0]
        input_features = [
            {model_input_name: feature[model_input_name]} for feature in features
        ]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(
            input_features, return_tensors="pt"
        )

#         if self.forward_attention_mask:
#             batch["attention_mask"] = torch.LongTensor(
#                 [feature["attention_mask"] for feature in features]
#             )

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(
#             labels_batch.attention_mask.ne(1), -100
#         )

#         # if bos token is appended in previous tokenization step,
#         # cut bos token here as it's append later anyways
#         if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
#             labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
#     decoder_start_token_id=model.config.decoder_start_token_id,
#     forward_attention_mask=forward_attention_mask,
)

In [ ]:
import torch #### WAC2VEC2

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [26]:
print("Train dataset length:", len(train_hf))
print("Eval dataset length:", len(eval_hf))
print("First sample keys:", train_hf[0].keys())

Train dataset length: 13663
Eval dataset length: 1708
First sample keys: dict_keys(['input_features', 'labels'])


In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=f"./openai-whisper-v3-milamou-lr3e-5-batch32/64",
    num_train_epochs=8,
    
    ################# 
    per_device_train_batch_size=8,        
    per_device_eval_batch_size=8,         
    gradient_accumulation_steps=6,       
    ################
    
    learning_rate=3e-5,
    warmup_steps=1500,
    
    #################### A100 
    gradient_checkpointing=True,        
    bf16=True,                           
    dataloader_pin_memory=True,        
    dataloader_num_workers=8,            
    #################
    
    save_steps=200,
    eval_steps=50,                      
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to='wandb',
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=50,                    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hf,
    eval_dataset=eval_hf,
    data_collator=data_collator,
    tokenizer=processor.tokenizer,
)

In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: elormiden (elormiden-university-of-central-lancashire) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


ValueError: Whisper expects the mel input features to be of length 3000, but found 1. Make sure to pad the input mel features to 3000.

In [42]:
trainer.save_model("./final-wac2vec2model")
processor.save_pretrained("./final-wac2vec2model")

[]

In [43]:
from transformers import AutoProcessor, AutoModelForCTC

trained_processor = AutoProcessor.from_pretrained("./final-wac2vec2model")
trained_model = AutoModelForCTC.from_pretrained("./final-wac2vec2model")

In [44]:
trained_model.push_to_hub("Elormiden/wac2vec2-milamou")
trained_processor.push_to_hub("Elormiden/wac2vec2-milamou")

CommitInfo(commit_url='https://huggingface.co/Elormiden/wac2vec2-milamou/commit/a27b107348534e11ce9b2cba96038166b782da89', commit_message='Upload processor', commit_description='', oid='a27b107348534e11ce9b2cba96038166b782da89', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Elormiden/wac2vec2-milamou', endpoint='https://huggingface.co', repo_type='model', repo_id='Elormiden/wac2vec2-milamou'), pr_revision=None, pr_num=None)